In [3]:
deployment = "gpt4"
model = "gpt-4"

In [2]:
from langchain.tools import ShellTool

shell_tool = ShellTool()

shell_tool.description = shell_tool.description + f"args {shell_tool.args}".replace(
    "{", "{{"
).replace("}", "}}")

print(shell_tool.description)


Run shell commands on this MacOS machine.args {{'commands': {{'title': 'Commands', 'description': 'List of shell commands to run. Deserialized using json.loads', 'anyOf': [{{'type': 'string'}}, {{'type': 'array', 'items': {{'type': 'string'}}}}]}}}}


In [12]:
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType

llm = AzureChatOpenAI(deployment_name = deployment, model_name=model, temperature=0, max_tokens=200)

shell_tool.description = shell_tool.description + f"args {shell_tool.args}".replace(
    "{", "{{"
).replace("}", "}}")
self_ask_with_search = initialize_agent(
    [shell_tool], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
self_ask_with_search.run(
    "list all the golang source code's file name in current directory and its sub directoris."
)



> Entering new AgentExecutor chain...
Thought: To list all the Golang source code files in the current directory and its subdirectories, we can use the `find` command in the terminal. The `find` command allows us to search for files in a directory hierarchy. We can specify the `-name` option to search for files with a specific pattern. In this case, we want to find all files with the `.go` extension, which are Golang source code files.

Action:
```
{
  "action": "terminal",
  "action_input": "find . -name '*.go'"
}
```
Observation: ./llm_friendly_code/demo/skeleton.go
./llm_friendly_code/demo/processor_rev.go
./llm_friendly_code/demo/token_gen_uuid.go
./llm_friendly_code/demo/grpc_endpoint.go
./llm_friendly_code/demo/skeleton_interfaces.go
./llm_friendly_code/demo/storage_map.go
./llm_friendly_code/demo/skeleton_test.go
./llm_friendly_code/endpoint/processing_service.pb.go
./llm_friendly_code/endpoint/processing_service_grpc.pb.go
./llm_friendly_code/main.go

Thought:The terminal com

'The Golang source code files in the current directory and its subdirectories are: skeleton.go, processor_rev.go, token_gen_uuid.go, grpc_endpoint.go'

# Zero-shot Learning

## 意图识别

In [4]:
import openai
def recognize_intent(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        temperature = 0,
        messages=[
            {"role": "system", "content": """
              Recognize the intent from the user's input and format output as JSON string. 
            The output JSON string includes: "intention", "paramters" """},
            {"role": "user", "content": input}
        ]
      )
    return response.choices[0].message.content

In [5]:
print(recognize_intent("订8月19日北京到上海的飞机"))

{"intention": "book_flight", "parameters": {"date": "8月19日", "departure": "北京", "destination": "上海"}}


## 情感分类

In [4]:
import openai

def classify_sentiment(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        messages=[
            {"role": "system", "content": """
             根据给定文字所表达的情感，将文字分为三类：正面 ,中立， 负面。请将用户给定的文字进行分类，并输出分类。
             """},   
            {"role": "user", "content": input}
        ],
        temperature = 0.9, 
        max_tokens = 200
      )
    return response.choices[0].message.content

In [5]:
classify_sentiment("通过前面的课程我对大模型的认识提高了很多。")

'正面'

In [6]:
classify_sentiment("今天是2023年8月16日")

'中立'

In [7]:
classify_sentiment("最近感觉很累。")

'负面'

# Few-shot Learning

In [19]:
import openai
def book_flight(input):
    response = openai.ChatCompletion.create(
        engine=deployment, # engine = "deployment_name".
        temperature = 0,
        messages=[
            {"role": "system", "content": """
              通过用户描述，提取航班预订信息并以JSON格式输出. 
              以下是一些示例：
              示例1
              输入: "订8月9日上海到北京的航班，上午出发"
              输出:"{"date":"0809","dispatch_time":"6-12","from":"shanghai","to":"beijing"}"
              
              示例2
              输入: "订8月9日上海到北京的航班，下午出发"
              输出:"{"date":"0809","dispatch_time":"12-18","from":"shanghai","to":"beijing"}"
              
              示例3
              输入: "订8月9日上海到北京的航班，晚上出发"
              输出:"{"date":"0809","dispatch_time":"18-24","from":"shanghai","to":"beijing"}"
              
              示例4
              输入: "订8月9日上海到北京的航班"
              输出:"{"date":"0809","dispatch_time":"unknown","from":"shanghai","to":"beijing"}"
              """},
            {"role": "user", "content": input}
        ]
      )
    return response.choices[0].message.content

In [20]:
book_flight("预订9月3日北京到广州的航班")

'{"date":"0903","dispatch_time":"unknown","from":"beijing","to":"guangzhou"}'

In [21]:
book_flight("预订北京到广州的航班")

'{"date":"unknown","dispatch_time":"unknown","from":"beijing","to":"guangzhou"}'

In [22]:
book_flight("预订9月3日夜里出发到广州的航班")

'{"date":"0903","dispatch_time":"18-24","from":"unknown","to":"guangzhou"}'